# Intro to Machine Learning Pipelines

<a target="_blank" href="https://colab.research.google.com/github/unionai-oss/intro-to-ml-pipelines/blob/main/intro-to-ml-pipelines.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Google Colab / Jupyter notebook

In [12]:
print("Hello, world!")

Hello, world!


# Workshop Setup

Sign up for Union while libraries are installing below:

- Union sign up: https://signup.union.ai/?group=workshop
- Union Platform: https://serverless.union.ai/
- Hugging Face Sign up: https://huggingface.co/

Install python libraries by running the code cell below:

In [ ]:
%%bash
pip install -U scikit-learn==1.4.1.post1 matplotlib==3.8.3 seaborn==0.13.2 union==0.1.48 keyrings.alt==5.0.0 huggingface_hub==0.24.0 joblib==1.3.2

Auth Union

In [7]:
!union create login --auth device-flow

Login successful!


## Build a Simple Workflow
- Task Docs
- Workflow Docs

Flyte

Union


In [ ]:
%%writefile simple_wf.py

# Import libraries and modules
# task
from flytekit import task, workflow

@task
def hello_world(name: str) -> str:
    return f"Hello {name}"

# workflow
@workflow
def main(name: str) -> str:
    return hello_world(name=name)


In [ ]:
# Run locally
!union run simple_wf.py main --name Flyte

In [ ]:
# Run on Union
!union run --remote simple_wf.py main --name Flyte

In [ ]:
!union run simple_wf.py main --help

In [ ]:
!union run --help

# Machine Learning Workflow

- Create a Container image
- Download Dataset
- Process Data & Visualize
- Train Machine Learning Model
- Evaluate Machine Learning Model
- Save Model to Hugging Face Model Hub

In [46]:
# Secrets in Union
# HF Secret
!union create secret hf_token

Enter secret value: 
Created secret with name: hf_token


In [ ]:
!union get secret

In [ ]:
# !union delete secret hf_token

In [61]:
%%writefile workflow.py

# Import libraries and modules
from textwrap import dedent
import html
from flytekit import task, workflow, ImageSpec, current_context, Deck, Resources, Secret
from flytekit.deck.renderer import MarkdownRenderer
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    ConfusionMatrixDisplay,
    accuracy_score,
    classification_report,
)
from sklearn.base import BaseEstimator
from sklearn.ensemble import HistGradientBoostingClassifier
import matplotlib.pyplot as plt
import matplotlib as mpl
import io
import base64
import seaborn as sns

from huggingface_hub import HfApi

from sklearn.datasets import load_iris
import joblib

# Define the container image to use for the tasks on Union with ImageSpec
image = ImageSpec(
    packages=[
        "scikit-learn==1.4.1.post1",
        "matplotlib==3.8.3",
        "union==0.1.48",
        "seaborn==0.13.2",
        "joblib==1.3.2",
        "huggingface_hub==0.24.0"
    ],
)

# Helper function to convert a matplotlib figure into an HTML string
def _convert_fig_into_html(fig: mpl.figure.Figure) -> str:
    img_buf = io.BytesIO()
    fig.savefig(img_buf, format="png")
    img_base64 = base64.b64encode(img_buf.getvalue()).decode()
    return f'<img src="data:image/png;base64,{img_base64}" alt="Rendered Image" />'


# Task: Download the dataset
@task(
    cache=True,
    cache_version="6",
    container_image=image,
    requests=Resources(cpu="2", mem="2Gi"),
)
def download_dataset() -> pd.DataFrame:
    iris = load_iris()
    iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
    iris_df['target'] = iris.target
    return iris_df

# Task: Process the dataset
@task(
    enable_deck=True,
    container_image=image,
    requests=Resources(cpu="2", mem="2Gi"),
)
def process_dataset(data_df: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:

    # Perform the train-test split
    train_df, test_df = train_test_split(data_df,
                                         test_size=0.2,
                                         random_state=42,
                                         stratify=data_df['target'])

    # Seaborn pairplot full dataset
    pairplot = sns.pairplot(data_df, hue="target")

    metrics_deck = Deck("Metrics")
    metrics_deck.append(_convert_fig_into_html(pairplot))

    return train_df, test_df


# Train a model using the HistGradientBoostingClassifier
@task(
    container_image=image,
    requests=Resources(cpu="3", mem="3Gi"),
)
def train_model(dataset: pd.DataFrame) -> BaseEstimator:
    X_train, y_train = dataset.drop("target", axis="columns"), dataset["target"]
    hist = HistGradientBoostingClassifier(
        random_state=0, categorical_features="from_dtype"
    )
    return hist.fit(X_train, y_train)

# Evaluate the model using the test dataset
@task(
    container_image=image,
    enable_deck=True,
    requests=Resources(cpu="2", mem="2Gi"),
)
def evaluate_model(model: BaseEstimator, dataset: pd.DataFrame) -> BaseEstimator:
    ctx = current_context()

    X_test, y_test = dataset.drop("target", axis="columns"), dataset["target"]
    y_pred = model.predict(X_test)

    # Plot confusion matrix in deck
    fig, ax = plt.subplots()
    ConfusionMatrixDisplay.from_predictions(y_test, y_pred, ax=ax)

    metrics_deck = Deck("Metrics")
    metrics_deck.append(_convert_fig_into_html(fig))

    # Add classification report
    report = html.escape(classification_report(y_test, y_pred))
    html_report = dedent(
        f"""\
    <h2>Classification report</h2>
    <pre>{report}</pre>"""
    )
    metrics_deck.append(html_report)

    ctx.decks.insert(0, metrics_deck)

    return model

# Upload the model to Hugging Face Hub
@task(
    container_image=image,
    requests=Resources(cpu="1", mem="1Gi"),
    secret_requests=[Secret(group=None, key="hf_token")],
)
def upload_model_to_hf(model: BaseEstimator, repo_name: str, model_name: str) -> str:
    ctx = current_context()
    hf_token = ctx.secrets.get(key="hf_token")
    # Create a new repository (if it doesn't exist)
    api = HfApi()
    api.create_repo(repo_name, token=hf_token, exist_ok=True)

    # save model
    joblib.dump(model, model_name)

    # Upload the model to the HF repository
    api.upload_file(
        path_or_fileobj=model_name,
        path_in_repo=model_name,
        repo_id=repo_name,
        commit_message="Upload model",
        repo_type=None,
        token=hf_token
    )
    return f"Model uploaded to Hugging Face Hub: {repo_name}{model_name}"

# Main workflow that orchestrates the tasks
@workflow
def main(repo_name: str, model_name: str) -> str:
    data_df = download_dataset()
    train, test = process_dataset(data_df)
    model = train_model(dataset=train)
    evaluated_model = evaluate_model(model=model, dataset=test)
    model_name = model_name
    upload_result = upload_model_to_hf(model=evaluated_model,
                                       repo_name=repo_name,
                                       model_name=model_name)
    return upload_result


Overwriting workflow.py


In [ ]:
! union run workflow.py main --help

In [ ]:
# Run workflow remotely (Union serverless) or your own compute
!union run --remote workflow.py main --repo_name sagecodes/test-upload5 --model_name model.pkl

View workflow in Union dashboard

## Download Model from Hugging Face Hub

In [ ]:
from huggingface_hub import hf_hub_download
hf_token = ""

hf_model = hf_hub_download(repo_id="YOURUSERNAME/REPOID",
                filename="model.pkl",
                token=hf_token)

model = joblib.load(hf_model)


In [ ]:
model

HistGradientBoostingClassifier(categorical_features='from_dtype',
                               random_state=0)

In [ ]:
model.predict([[3,5,6,5]])
model.predict([[.4,.2,.1,.6]])

Keep Learning:

